# Importing libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import string 

# Declare common global object

In [2]:
completeUrl = []
# Change letter=? , which letter detail you want to scrap
plantes_by_Alphabet = 'letter=Z'
pagenoword ='Showing 100 of '
data_plant = None
keywords =['root','stem','leave','fruit','latex','bark' ,'pulp', 'flower' ,'whole plant' ,
           'seed','exudate','nuts','sap','rhizome','bulb','twig','pod','wood','tuber']

# Custom funtion to find the index of word in text

In [3]:
def using_str_index_lst (word, sentence):
    lst = []
    index = 0;
    count = 0;
    while index < len(sentence):
            index = sentence.find(word, index)
            count = count + 1;
            if index == -1:
                break
            lst.append(index)
            index += len(word)
    return lst


# Custom function to remove special characters

In [4]:
rmPunctuation = string.punctuation 
rmPunctuation.replace('.','')

def remove_punctuation(text):
    result = []
    file = text
    text= text.replace('<br>','')
    text= text.replace('\n','')
    text= text.replace('\r','')
    text= text.replace('$',' ')
    for line in file:
         for char in line:
            if char not in rmPunctuation:
                result.append(char)
    result = ''.join(result)
    result.replace('br','')
    return result

# Custom funtion to find keywords in text and return found keywords

In [5]:
def find_word_retun_allword(keywords,text):
    foundword = ''
    for word in keywords:
        if text.find(word) != -1 :
            foundword += ',' + word
    
    return foundword[1:]

# Custom function to find page number to navigate 

In [6]:
def find_page_count(text,searchtext):
    pageno = 0
    pagenumber =[]
    searchtextlen = len(searchtext)
    startpoint = using_str_index_lst('Showing 100 of ',text)
    if len(startpoint) > 0:
        startpoint = startpoint[0]+searchtextlen
        endpoint = using_str_index_lst(' ',text[startpoint:])
        endpoint = endpoint[0]
        pagenumber = text[startpoint:startpoint+endpoint]
    if len(pagenumber) > 0:
        pageno = int(pagenumber)
    else:
        pageno = 0
    return pageno

In [7]:

URL = 'https://tropical.theferns.info/letter-index.php?letter=A&amount=100&pageno=0'
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
URL =URL.replace('letter=A',plantes_by_Alphabet)
r = requests.get(url=URL, headers=headers)

In [8]:
soup = BeautifulSoup(r.content, 'html.parser')# 'html5lib')

In [9]:
a_tag = soup.find_all("a")

In [10]:
urllist = []
for tag in a_tag:
    urllist.append(tag.get('href'))

In [11]:
if None in urllist:
    urllist.remove(None)

In [12]:
subs = 'viewtropical'
urllist = [i for i in urllist if subs in i]

In [13]:
for item in urllist:
    item = 'https://tropical.theferns.info/'+item
    completeUrl.append(item)

In [14]:
maxpage = find_page_count(str(soup),pagenoword)
# maxpage

In [15]:
for page in range(100,maxpage,100):
    URL = 'https://tropical.theferns.info/letter-index.php?letter=A&amount=100&pageno=0'
    URL =URL.replace('letter=A',plantes_by_Alphabet)
    URL = URL.replace('pageno=0','pageno='+str(page))
    r = requests.get(URL, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')# 'html5lib')
    a_tag = soup.find_all("a")
    urllist = []
    for tag in a_tag:
        urllist.append(tag.get('href'))
    if None in urllist:
        urllist.remove(None)
    subs = 'viewtropical'
    urllist = [i for i in urllist if subs in i]
    for item in urllist:
        item = 'https://tropical.theferns.info/'+item
        completeUrl.append(item)
    

# Custom method for webscaping for specific url and returns dataframe

In [16]:
def scrap_data_web(url,data_plant):
    URL = url
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
    r = requests.get(url=URL, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')# 'html5lib')
    latin_name = soup.find_all("div",class_ = 'latin_name')
    latin_name = soup.find_all('h1')
    family = soup.find_all('h4')
    Medicinal = soup.find_all('h3')
    Species_Name = latin_name[0].text
    family_name = family[1].text
    common_name = family[2].text.replace('Common Name:','')
    textpage=soup
    textpage = str(textpage)
    start_point = using_str_index_lst('<h3>Medicinal', textpage)
    
    if len(start_point) != 0:
    
        if len(using_str_index_lst('Agroforestry Uses:', textpage)) == 0:
            end_point = using_str_index_lst('<h3>Other Uses', textpage)
        else:
            end_point = using_str_index_lst('Agroforestry Uses:', textpage)

        textpage = textpage[start_point[0]:end_point[0]]
        remove_start = using_str_index_lst('[', textpage)
        remove_end = using_str_index_lst(']', textpage)
        if(len(remove_start)== len(remove_end)):
            for i in range(len(remove_start)):
                textpage = textpage.replace(textpage[remove_start[i]:remove_end[i]+1], '$'*(remove_end[i]+1-remove_start[i]))

            Medicinal_Use = remove_punctuation(textpage[17:])
        else:
            Medicinal_Use = ''
    else:
        Medicinal_Use = ''
        
        Medicinal_Use.replace('br\n','')   
        
    data = []
    local_name = ''
    parts_used = ''
    parts_used = find_word_retun_allword(keywords,Medicinal_Use.lower ())
    data = [Species_Name,family_name,local_name,common_name.strip(),parts_used,Medicinal_Use]
    
    d = {'Species_Name': [data[0]], 'Family_name':  [data[1]], 'Local_name':  local_name, 'Common_name': [data[3] ],'Parts_used':parts_used,'Medicinal_Use':[data[5]] }
    ndf = pd.DataFrame(d)

    return ndf

# Calling webscraping method for each url

In [17]:
for url in completeUrl:
    d = scrap_data_web(url,data_plant)
    if data_plant is not None:
        data_plant = data_plant.append(d, ignore_index = True )
    else:
        data_plant = pd.DataFrame(d)
    

# Clean dataframe

In [18]:
data_plant['Medicinal_Use'] = data_plant['Medicinal_Use'].str.replace('None knownbr\n','None known')

In [19]:
data_plant

,Species_Name,Family_name,Local_name,Common_name,Parts_used,Medicinal_Use
0,Zaleya pentandra,Aizoaceae,,,sap,The aboveground parts of the plant are used as...
1,Zamia angustifolia,Zamiaceae,,Yuquilla de Paredón,,None known
2,Zamia chigua,Zamiaceae,,Chigua Macho,,None known
3,Zamia erosa,Zamiaceae,,,,None known
4,Zamia integrifolia,Zamiaceae,,Florida Arrowroot,,None knownh3
...,...,...,...,...,...,...
80,Zornia leptophylla,Fabaceae,,,,None known
81,Zornia piurensis,Fabaceae,,Hierba de la Vibora,whole plant,A decoction of the whole plant combined with C...
82,Zuelania guidonia,Salicaceae,,,"stem,bark",A gum obtained from the stem is emeticbr\r\nTh...
83,Zygia latifolia,Fabaceae,,,bark,The inner bark rasped is in a cold water infus...


# Create csv and json file from dataframe

In [20]:
data_plant.to_csv('data_plant'+plantes_by_Alphabet+'.csv')
data_plant.to_json('data_plant'+plantes_by_Alphabet+'.json',orient ='records')